In [ ]:
import csv 
import json 

def csv_to_json(csvFilePath, jsonFilePath):
    jsonArray = []
      
    #read csv file
    with open(csvFilePath, encoding='utf-8') as csvf: 
        #load csv file data using csv library's dictionary reader
        csvReader = csv.DictReader(csvf) 

        #convert each csv row into python dict
        for row in csvReader: 
            #add this python dict to json array
            jsonArray.append(row)
  
    #convert python jsonArray to JSON String and write to file
    with open(jsonFilePath, 'w', encoding='utf-8') as jsonf: 
        jsonString = json.dumps(jsonArray, indent=4)
        jsonf.write(jsonString)
          
csvFilePath = r'data.csv'
jsonFilePath = r'data.json'
csv_to_json('data/databern.csv', 'data/bern.json')


In [8]:
#import piplite
#await piplite.install('ipywidgets')
#await piplite.install('pandas')
#await piplite.install('matplotlib')
#await piplite.install('geopy')
#await piplite.install('pivottablejs')
#await piplite.install('ipyleaflet')
#await piplite.install('gender_guesser')
!pip install pandas

import pandas as pd

from utils.prepare_data import getJSON

     ---------------------------------------- 10.9/10.9 MB 1.7 MB/s eta 0:00:00
  Using cached numpy-1.23.4-cp39-cp39-win_amd64.whl (14.7 MB)


You should consider upgrading via the 'C:\Users\alecroq\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [9]:
data = getJSON('data/bern.json')

df = pd.json_normalize(data[200:220])
pd.DataFrame(df)

,ï»¿Link,Sprache,Erscheinungsort,Jahr
0,[On the effects of oxygen in accelerating germ...,Englisch,"Boston, Massachusetts",1802
1,[Ueber den Pico de Teyde auf Teneriffa](https:...,Deutsch,Freiberg,1802
2,[F. Humboldtâs Brief an Hrn. Professor Picte...,Deutsch,Leipzig,1802
3,"[Lettre de F. Humboldt Ã M. Pictet, professeu...",FranzÃ¶sisch,Paris,1802
4,[Extract of a letter from M. Von Humboldt to L...,Englisch,London,1802
5,[Skizze einer Geologischen Schilderung des sÃŒ...,Deutsch,Weimar,1802
6,[Copie dâune lettre lue Ã la Classe des Sci...,FranzÃ¶sisch,Paris,1803
7,"[Alexandre Humboldt au Citoyen Delambre, SecrÃ...",FranzÃ¶sisch,London,1803
8,[Kurzer Auszug aus Hrn. Alexand. v. Humboldtâ...,Deutsch,Weimar,1803
9,[[Brief an Wilhelm von Humboldt]](https://humb...,Deutsch,Weimar,1803


In [79]:
import geocoder

def getGeolocalisationPlace(place):
    """
    Give the geolocation of a giving place.
    The geolocalisation is search in differents local files :
    - location register of edition humboldt
    - location register from GeoName
    If the place isn't already stored in one of this file, then
    a request is sent to the GeoName API.
    :param place: str
    :return: coverage_location
    :rtype: dict
    """
    ortsregister = getJSON('data/edh_ortsregister.json')
    or_geoname = getJSON('data/geoname_ortsregister.json')
    
    coverage_location = {}
    from_geoname = {}
    try :
        for o in ortsregister:
            # If the place of the ortsregister of the edh is the same of the giving place
            # then add the diverse propreties of ortsregister's place to the dict
            # coverage_location
            if place == o['properties']['ContentHeader'] :
                coverage_location["key"] = o['properties']['key']
                coverage_location['geoname_id'] = o['properties']['geoname_id']
                coverage_location['address'] = o['properties']['ContentHeader']
                coverage_location['coordinates'] = o['geometry']['coordinates']
    
        # If the is no place in the ortsregister equal of the giving place
        # then the coverage_location dict stayed empty and we check if the place
        # already was stored in the geoname_ortsregister
        if bool(coverage_location)== False :
            for o in or_geoname:
                for i in o:
                    if place == o[i]["address"]:
                        try:
                            coverage_location["key"] = o[i]['key']
                        except: pass
                        coverage_location['geoname_id'] = o[i]['geoname_id']
                        coverage_location['address'] = i
                        coverage_location['coordinates'] = o[i]['coordinates']

        # If the giving place is neither in the ortsregister of the edh and in
        # stored places from geoname file, then we send a request to geoname to
        # get a geolocation      
        if bool(coverage_location)== False :
            location = geocoder.geonames(place, key='dumont', featureClass='P')
            coverage_location['geoname_id'] = str(location.geonames_id)
            coverage_location['address'] = location.address
            coverage_location['coordinates'] = [location.lng, location.lat]
        
            for ort in ortsregister:
                if ort['properties']['geoname_id'] == str(coverage_location['geoname_id']):
                    coverage_location['key'] = ort['properties']['key']
            from_geoname[place] = coverage_location
        
        # If some geolocalisation from geoname weren't already 
        # stored in some local file, then they are added in the
        # proper geoname_ortsregister.json
        if bool(from_geoname) == True:
            d = getJSON("data/geoname_ortsregister.json")
            d.append(from_geoname)            
            writeJSON("data/geoname_ortsregister.json", d)
    except :
        print(place)
    
    return coverage_location

In [78]:
#getGeolocalisationPlace("Brussel")
or_geoname = getJSON('data/geoname_ortsregister.json')
coverage_location = {}
place = 'Percy-en-Auge'
test= {"Freiburg": {"geoname_id": "2925177", "address": "Freiburg im Breisgau", "coordinates": ["7.85222", "47.9959"], "key": "H0004809"}}, {"Leamington": {"geoname_id": "2644737", "address": "Royal Leamington Spa", "coordinates": ["-1.52", "52.2852"]}}, {"Plymouth": {"geoname_id": "2186239", "address": "New Plymouth", "coordinates": ["174.08333", "-39.06667"]}}, {"Vlissingen": {"geoname_id": "2745392", "address": "Flushing", "coordinates": ["3.57361", "51.4425"]}}, {"Percy-en-Auge": {"geoname_id": "2988016", "address": "Percy-en-Auge", "coordinates": ["-0.06354", "49.05312"]}}, {"Brussel": {"geoname_id": "2800866", "address": "Brussels", "coordinates": ["4.34878", "50.85045"], "key": "H0005371"}}

for o in test:
                for i in o:
                    if place == o[i]["address"]:
                        try:
                            coverage_location["key"] = o[i]['key']
                        except: pass
                        coverage_location['geoname_id'] = o[i]['geoname_id']
                        coverage_location['address'] = i
                        coverage_location['coordinates'] = o[i]['coordinates']

print(coverage_location)

{'geoname_id': '2988016', 'address': 'Percy-en-Auge', 'coordinates': ['-0.06354', '49.05312']}


In [50]:
!pip install geocoder

     ---------------------------------------- 98.6/98.6 KB 2.8 MB/s eta 0:00:00
     ---------------------------------------- 96.6/96.6 KB 2.8 MB/s eta 0:00:00
     -------------------------------------- 829.2/829.2 KB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
Using legacy 'setup.py install' for future, since package 'wheel' is not installed.
  Running setup.py install for future: started
  Running setup.py install for future: finished with status 'done'


You should consider upgrading via the 'C:\Users\alecroq\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [87]:
import json

def writeJSON(file, data):
    """
    Store JSON data in a JSON file 
    :param file: str
    :param data: dict
    """
    with open(file, mode='w') as f:
        json.dump(data, f)

dataWithGeo= []
index = 0
data = getJSON('data/bern.json')

for i in data:
    item = {}
    item['link'] = data[index]['ï»¿Link'].split('(')[1].split(')')[0]
    item['title'] = data[index]['ï»¿Link'].split('[')[1].split(']')[0] # ï»¿Link

    if ',' in data[index]['Erscheinungsort']:
        place = data[index]['Erscheinungsort'].split(',')[0]
    elif ';' in data[index]['Erscheinungsort']:
        place = data[index]['Erscheinungsort'].split(';')[0]
    else : place = data[index]['Erscheinungsort']
    item['pubplace'] = getGeolocalisationPlace(place)
    item['year'] = data [index]['Jahr']
    item['language'] = data[index]['Sprache']
    index += 1
    dataWithGeo.append(item)

writeJSON('data/bern_withgeo.json', dataWithGeo)


Error the hourly limit of 1000 credits for dumont has been exceeded. Please throttle your requests or use the commercial service. from JSON {'status': {'message': 'the hourly limit of 1000 credits for dumont has been exceeded. Please throttle your requests or use the commercial service.', 'value': 19}}
Error the hourly limit of 1000 credits for dumont has been exceeded. Please throttle your requests or use the commercial service. from JSON {'status': {'message': 'the hourly limit of 1000 credits for dumont has been exceeded. Please throttle your requests or use the commercial service.', 'value': 19}}
Error the hourly limit of 1000 credits for dumont has been exceeded. Please throttle your requests or use the commercial service. from JSON {'status': {'message': 'the hourly limit of 1000 credits for dumont has been exceeded. Please throttle your requests or use the commercial service.', 'value': 19}}
Error the hourly limit of 1000 credits for dumont has been exceeded. Please throttle you

KeyboardInterrupt: 

In [25]:
!pip install ipyleaflet

     ---------------------------------------- 3.7/3.7 MB 2.0 MB/s eta 0:00:00
  Using cached xyzservices-2022.9.0-py3-none-any.whl (55 kB)
  Using cached ipywidgets-8.0.2-py3-none-any.whl (134 kB)
  Using cached traittypes-0.2.1-py2.py3-none-any.whl (8.6 kB)
  Using cached branca-0.5.0-py3-none-any.whl (24 kB)
  Using cached jupyterlab_widgets-3.0.3-py3-none-any.whl (384 kB)
  Using cached widgetsnbextension-4.0.3-py3-none-any.whl (2.0 MB)


You should consider upgrading via the 'C:\Users\alecroq\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [89]:
geo = getJSON('data/bern_withgeo.json')
allonmap_bern(geo)


NameError: name 'allonmap_bern' is not defined